In [25]:
import pandas as pd
import numpy as np
from scipy.fft import fft, fftfreq

In [37]:
# Example of loading the data into a DataFrame
df = pd.read_csv('movement_file.csv')

In [39]:
df.head()

,id,file,Number of time steps,Relaxed1_Left_Accelerometer_X,Relaxed1_Left_Accelerometer_Y,Relaxed1_Left_Accelerometer_Z,Relaxed1_Left_Gyroscope_X,Relaxed1_Left_Gyroscope_Y,Relaxed1_Left_Gyroscope_Z,Relaxed1_Right_Accelerometer_X,...,Entrainment2_Left_Accelerometer_Z,Entrainment2_Left_Gyroscope_X,Entrainment2_Left_Gyroscope_Y,Entrainment2_Left_Gyroscope_Z,Entrainment2_Right_Accelerometer_X,Entrainment2_Right_Accelerometer_Y,Entrainment2_Right_Accelerometer_Z,Entrainment2_Right_Gyroscope_X,Entrainment2_Right_Gyroscope_Y,Entrainment2_Right_Gyroscope_Z
0,1,001_ml.bin,1,-0.007183,-0.007979,-0.007813,-0.006712,-0.007526,-0.006415,-0.008140,...,0.000559,0.001522,0.000559,0.001519,-0.002423,-0.002487,-0.000593,0.000311,-0.001722,-0.000819
1,1,001_ml.bin,2,0.001043,0.001997,0.002999,0.003058,0.001165,-0.000729,0.001272,...,0.001436,0.004418,0.001537,0.000647,0.000701,-0.000202,-0.000140,-0.002978,-0.003918,-0.003884
2,1,001_ml.bin,3,-0.001907,0.000037,0.001010,0.002977,0.002096,0.000236,0.000322,...,-0.004281,-0.006298,-0.006403,-0.004579,-0.000845,0.000921,0.000764,0.003550,0.003449,0.003356
3,1,001_ml.bin,4,0.002346,0.002297,0.002318,0.003289,-0.000583,0.001389,0.003343,...,-0.001967,-0.000019,-0.001957,-0.001020,-0.003008,-0.002099,-0.001175,-0.000244,-0.000305,0.000652
4,1,001_ml.bin,5,-0.000287,-0.000247,-0.001204,-0.003151,-0.003194,-0.000343,-0.000421,...,-0.002292,-0.000363,-0.000385,0.001572,0.003541,0.004588,0.005647,0.004855,0.003061,0.001238


In [41]:
df.rename(columns={'id': 'ID'}, inplace=True)

In [43]:
df.isna().sum()

ID                                    0
file                                  0
Number of time steps                  0
Relaxed1_Left_Accelerometer_X         0
Relaxed1_Left_Accelerometer_Y         0
                                     ..
Entrainment2_Right_Accelerometer_Y    0
Entrainment2_Right_Accelerometer_Z    0
Entrainment2_Right_Gyroscope_X        0
Entrainment2_Right_Gyroscope_Y        0
Entrainment2_Right_Gyroscope_Z        0
Length: 135, dtype: int64

In [21]:
# Function to assess kinetic tremor based on amplitude
def assess_kinetic_tremor(amplitude_value):
    """Assess kinetic tremor based on amplitude value during the finger-to-nose maneuver"""
    if amplitude_value == 0:
        return 0  # Normal: No tremor
    elif amplitude_value < 1:
        return 1  # Slight: Tremor is less than 1 cm in amplitude
    elif amplitude_value < 3:
        return 2  # Mild: Tremor is between 1-3 cm
    elif amplitude_value < 10:
        return 3  # Moderate: Tremor is between 3-10 cm
    else:
        return 4  # Severe: Tremor is at least 10 cm

# Function to assess rest tremor based on amplitude
def assess_rest_tremor(amplitude_value):
    """Assess rest tremor based on amplitude value observed while the patient is sitting quietly"""
    if amplitude_value == 0:
        return 0  # Normal: No tremor
    elif amplitude_value < 1:
        return 1  # Slight: Tremor is less than 1 cm in amplitude
    elif amplitude_value < 3:
        return 2  # Mild: Tremor is between 1-3 cm
    elif amplitude_value < 10:
        return 3  # Moderate: Tremor is between 3-10 cm
    else:
        return 4  # Severe: Tremor is at least 10 cm

# Modify the process_patient_data function
def process_patient_data(patient_data):
    # Initialize an empty dictionary to store results for each patient
    results = {}
    
    # Get the patient ID from the 'ID' column
    patient_id = patient_data['ID']
    
    # List of tasks and sensor types to loop through
    tasks = ['Relaxed1', 'Relaxed2', 'RelaxedTask1', 'RelaxedTask2', 'StretchHold', 
             'HoldWeight', 'DrinkGlas', 'CrossArms', 'TouchNose', 'Entrainment1', 'Entrainment2']
    sides = ['Left', 'Right']
    sensors = ['Accelerometer', 'Gyroscope']
    axes = ['X', 'Y', 'Z']
    
    # Add task categories for Postural, Kinetic, and Rest categories
    kinetic_tasks = ['TouchNose', 'CrossArms', 'DrinkGlas', 'Entrainment1', 'Entrainment2']
    rest_tasks = ['Relaxed1', 'Relaxed2', 'RelaxedTask1', 'RelaxedTask2', 'StretchHold', 'HoldWeight']
    
    # Loop over all the tasks, sides, sensors, and axes to extract data and calculate features
    for task in tasks:
        for side in sides:
            for sensor in sensors:
                for axis in axes:
                    # Construct the column name dynamically
                    column_name = f"{task}_{side}_{sensor}_{axis}"
                    
                    # Check if the column exists in the patient data
                    if column_name in patient_data:
                        signal = patient_data[column_name]
                        
                        # Only calculate features if the signal is not a scalar (float)
                        if isinstance(signal, pd.Series):
                            # Calculate the features (RMS, STD, Dominant Frequency, etc.)
                            rms_value = calculate_rms(signal)
                            std_value = calculate_std(signal)
                            freq_value = calculate_dominant_frequency(signal, fs=100)  # Assuming 100 Hz sampling frequency
                            amplitude_value = calculate_amplitude(rms_value)
                            
                            # For kinetic tasks, calculate the tremor rating based on amplitude
                            if task in kinetic_tasks:
                                tremor_rating = assess_kinetic_tremor(amplitude_value)
                                results[f"{task}_{side}_{sensor}_{axis}_KineticTremorRating"] = tremor_rating
                            # For rest tasks, calculate the tremor rating based on amplitude
                            elif task in rest_tasks:
                                tremor_rating = assess_rest_tremor(amplitude_value)
                                results[f"{task}_{side}_{sensor}_{axis}_RestTremorRating"] = tremor_rating
                            
                            # Store the rest of the results (RMS, STD, Frequency, Amplitude)
                            results[f"{task}_{side}_{sensor}_{axis}_RMS"] = rms_value
                            results[f"{task}_{side}_{sensor}_{axis}_STD"] = std_value
                            results[f"{task}_{side}_{sensor}_{axis}_Frequency"] = freq_value
                            results[f"{task}_{side}_{sensor}_{axis}_Amplitude"] = amplitude_value
    
    # Add the patient ID to the results
    results['ID'] = patient_id
    
    # Return the dictionary of results
    return results

# Apply the function to each row in the DataFrame
patient_results = df.apply(process_patient_data, axis=1)

# Convert the results into a more manageable format, e.g., a DataFrame
result_df = pd.DataFrame(patient_results.tolist())

# Optionally, add the 'ID' column back to the result DataFrame (if not already added)
result_df['ID'] = df['ID']

# View the result
print(result_df)


         ID
0         1
1         1
2         1
3         1
4         1
...     ...
457739  469
457740  469
457741  469
457742  469
457743  469

[457744 rows x 1 columns]


In [23]:
df.columns.to_list()

['ID',
 'file',
 'Number of time steps',
 'Relaxed1_Left_Accelerometer_X',
 'Relaxed1_Left_Accelerometer_Y',
 'Relaxed1_Left_Accelerometer_Z',
 'Relaxed1_Left_Gyroscope_X',
 'Relaxed1_Left_Gyroscope_Y',
 'Relaxed1_Left_Gyroscope_Z',
 'Relaxed1_Right_Accelerometer_X',
 'Relaxed1_Right_Accelerometer_Y',
 'Relaxed1_Right_Accelerometer_Z',
 'Relaxed1_Right_Gyroscope_X',
 'Relaxed1_Right_Gyroscope_Y',
 'Relaxed1_Right_Gyroscope_Z',
 'Relaxed2_Left_Accelerometer_X',
 'Relaxed2_Left_Accelerometer_Y',
 'Relaxed2_Left_Accelerometer_Z',
 'Relaxed2_Left_Gyroscope_X',
 'Relaxed2_Left_Gyroscope_Y',
 'Relaxed2_Left_Gyroscope_Z',
 'Relaxed2_Right_Accelerometer_X',
 'Relaxed2_Right_Accelerometer_Y',
 'Relaxed2_Right_Accelerometer_Z',
 'Relaxed2_Right_Gyroscope_X',
 'Relaxed2_Right_Gyroscope_Y',
 'Relaxed2_Right_Gyroscope_Z',
 'RelaxedTask1_Left_Accelerometer_X',
 'RelaxedTask1_Left_Accelerometer_Y',
 'RelaxedTask1_Left_Accelerometer_Z',
 'RelaxedTask1_Left_Gyroscope_X',
 'RelaxedTask1_Left_Gyroscope_Y

In [31]:
import pandas as pd
import numpy as np
from scipy.fft import fft, fftfreq

# Example to process one specific movement, e.g., Relaxed1_Left_Accelerometer_X
def calculate_rms(signal):
    """Calculate RMS of the signal."""
    return (signal**2).mean()**0.5

def calculate_std(signal):
    """Calculate standard deviation of the signal."""
    return signal.std()

def calculate_dominant_frequency(signal, fs=100):
    """Calculate dominant frequency of the signal using FFT."""
    N = len(signal)
    yf = fft(signal)
    xf = fftfreq(N, 1 / fs)
    return abs(xf[yf.argmax()])

def assess_kinetic_tremor(amplitude_value):
    """Assess kinetic tremor based on amplitude value during the finger-to-nose maneuver."""
    if amplitude_value == 0:
        return 0  # Normal: No tremor
    elif amplitude_value < 1:
        return 1  # Slight: Tremor is less than 1 cm in amplitude
    elif amplitude_value < 3:
        return 2  # Mild: Tremor is between 1-3 cm
    elif amplitude_value < 10:
        return 3  # Moderate: Tremor is between 3-10 cm
    else:
        return 4  # Severe: Tremor is at least 10 cm

def assess_rest_tremor(amplitude_value):
    """Assess rest tremor based on amplitude value observed while the patient is sitting quietly."""
    if amplitude_value == 0:
        return 0  # Normal: No tremor
    elif amplitude_value < 1:
        return 1  # Slight: Tremor is less than 1 cm in amplitude
    elif amplitude_value < 3:
        return 2  # Mild: Tremor is between 1-3 cm
    elif amplitude_value < 10:
        return 3  # Moderate: Tremor is between 3-10 cm
    else:
        return 4  # Severe: Tremor is at least 10 cm

def process_movement(df, movement_name, side, sensor_type, axis):
    # Construct the column name based on the movement and sensor details
    column_name = f"{movement_name}_{side}_{sensor_type}_{axis}"
    
    # Check if the column exists in the DataFrame
    if column_name in df.columns:
        signal = df[column_name].iloc[0]  # Get the first row's signal as a Series
        
        # Ensure the signal is a numpy array
        if isinstance(signal, pd.Series):
            signal = signal.to_numpy()

        # Calculate the features for the movement
        rms_value = calculate_rms(signal)
        std_value = calculate_std(signal)
        freq_value = calculate_dominant_frequency(signal, fs=100)  # Assuming 100 Hz sampling frequency
        amplitude_value = rms_value  # Using RMS as a proxy for amplitude
        
        # Assess kinetic or rest tremor based on task
        if movement_name in ['Relaxed1', 'Relaxed2', 'RelaxedTask1', 'RelaxedTask2', 'StretchHold']:
            tremor_rating = assess_rest_tremor(amplitude_value)
        else:
            tremor_rating = assess_kinetic_tremor(amplitude_value)

        # Return the calculated results for the movement
        return {
            f"{movement_name}_{side}_{sensor_type}_{axis}_RMS": rms_value,
            f"{movement_name}_{side}_{sensor_type}_{axis}_STD": std_value,
            f"{movement_name}_{side}_{sensor_type}_{axis}_Frequency": freq_value,
            f"{movement_name}_{side}_{sensor_type}_{axis}_Amplitude": amplitude_value,
            f"{movement_name}_{side}_{sensor_type}_{axis}_TremorRating": tremor_rating
        }
    else:
        # If the column doesn't exist, print an error message
        print(f"Column {column_name} not found in the DataFrame.")
        return {}

# Example DataFrame (df) with a sample row (just for demonstration)
# Assuming your df has a column structure as given in the question, 
# here is an example row for `Relaxed1_Left_Accelerometer_X`
sample_data = {
    'ID': [1],
    'Relaxed1_Left_Accelerometer_X': [np.random.randn(1024)],  # Random data for the sake of example
}

# Create a DataFrame from the sample data
df = pd.DataFrame(sample_data)

# Process a specific movement: Relaxed1_Left_Accelerometer_X
movement_name = "Relaxed1"
side = "Left"
sensor_type = "Accelerometer"
axis = "X"

# Call the process_movement function for this specific example
result = process_movement(df, movement_name, side, sensor_type, axis)

# Convert the result into a DataFrame to display the result
result_df = pd.DataFrame([result])

# Display the result DataFrame
result_df


,Relaxed1_Left_Accelerometer_X_RMS,Relaxed1_Left_Accelerometer_X_STD,Relaxed1_Left_Accelerometer_X_Frequency,Relaxed1_Left_Accelerometer_X_Amplitude,Relaxed1_Left_Accelerometer_X_TremorRating
0,1.037346,1.037292,0.683594,1.037346,2


In [45]:
def calculate_rms(signal):
    """Calculate the Root Mean Square (RMS) value of the signal."""
    return np.sqrt(np.mean(np.square(signal)))

def calculate_std(signal):
    """Calculate standard deviation of the signal."""
    return np.std(signal)

def calculate_dominant_frequency(signal, fs=100):
    """Calculate the dominant frequency of the signal using FFT."""
    n = len(signal)
    freqs = np.fft.fftfreq(n, 1/fs)
    fft_vals = fft(signal)
    positive_freqs = freqs[:n//2]
    positive_fft_vals = np.abs(fft_vals[:n//2])
    dominant_freq = positive_freqs[np.argmax(positive_fft_vals)]
    return dominant_freq

def calculate_tremor_rating(amplitude):
    """Calculate the tremor rating based on the amplitude."""
    if amplitude < 1:
        return 0  # Normal
    elif amplitude < 3:
        return 1  # Slight
    elif amplitude < 10:
        return 2  # Mild
    elif amplitude < 20:
        return 3  # Moderate
    else:
        return 4  # Severe

def process_movement(df, movement_name, side, sensor_type, axis):
    """Process a single movement (e.g., Relaxed1, StretchHold) for a given side, sensor, and axis."""
    # Construct the column name for the movement
    column_name = f"{movement_name}_{side}_{sensor_type}_{axis}"
    
    # Extract the signal (assuming the columns are already in the dataframe)
    signal = df[column_name].values
    
    # Calculate the features
    rms_value = calculate_rms(signal)
    std_value = calculate_std(signal)
    freq_value = calculate_dominant_frequency(signal, fs=100)  # Assuming 100 Hz sampling frequency
    amplitude_value = rms_value  # Using RMS as a proxy for amplitude
    tremor_rating = calculate_tremor_rating(amplitude_value)
    
    # Return the result as a dictionary
    return {
        "Movement": movement_name,
        "Side": side,
        "Sensor": sensor_type,
        "Axis": axis,
        "RMS": rms_value,
        "STD": std_value,
        "Frequency": freq_value,
        "Amplitude": amplitude_value,
        "TremorRating": tremor_rating
    }

def process_all_patients(df):
    """Process all patients (IDs 1 to 469) and all movements."""
    results = []

    # Define the movements, sides, sensor types, and axes
    movements = ['Relaxed1', 'Relaxed2', 'RelaxedTask1', 'RelaxedTask2', 'StretchHold', 'HoldWeight', 'DrinkGlas', 'CrossArms', 'TouchNose', 'Entrainment1', 'Entrainment2']
    sides = ['Left', 'Right']
    sensor_types = ['Accelerometer', 'Gyroscope']
    axes = ['X', 'Y', 'Z']
    
    # Loop through each patient ID (assuming 'ID' is the column for patient IDs)
    for patient_id in df['ID'].unique():
        patient_data = df[df['ID'] == patient_id]
        
        # Loop through each movement, side, sensor, and axis
        for movement in movements:
            for side in sides:
                for sensor_type in sensor_types:
                    for axis in axes:
                        # Process each movement and collect the results
                        result = process_movement(patient_data, movement, side, sensor_type, axis)
                        result['ID'] = patient_id  # Add the patient ID to the result
                        results.append(result)
    
    # Convert the results into a DataFrame
    result_df = pd.DataFrame(results)
    return result_df

# Example usage
# Assuming 'df' is your DataFrame with the necessary columns

result_df = process_all_patients(df)
print(result_df)


           Movement   Side         Sensor Axis       RMS       STD  Frequency  \
0          Relaxed1   Left  Accelerometer    X  0.669875  0.669874  16.290984   
1          Relaxed1   Left  Accelerometer    Y  0.666545  0.666544  16.290984   
2          Relaxed1   Left  Accelerometer    Z  0.661365  0.661363  16.290984   
3          Relaxed1   Left      Gyroscope    X  0.652301  0.652300  16.290984   
4          Relaxed1   Left      Gyroscope    Y  0.648110  0.648110  15.778689   
...             ...    ...            ...  ...       ...       ...        ...   
61903  Entrainment2  Right  Accelerometer    Y  0.509812  0.509802  11.372951   
61904  Entrainment2  Right  Accelerometer    Z  0.519732  0.519716  11.372951   
61905  Entrainment2  Right      Gyroscope    X  0.526807  0.526778  11.475410   
61906  Entrainment2  Right      Gyroscope    Y  0.527520  0.527478  11.475410   
61907  Entrainment2  Right      Gyroscope    Z  0.521691  0.521638  11.475410   

       Amplitude  TremorRat

In [47]:
result_df

,Movement,Side,Sensor,Axis,RMS,STD,Frequency,Amplitude,TremorRating,ID
0,Relaxed1,Left,Accelerometer,X,0.669875,0.669874,16.290984,0.669875,0,1
1,Relaxed1,Left,Accelerometer,Y,0.666545,0.666544,16.290984,0.666545,0,1
2,Relaxed1,Left,Accelerometer,Z,0.661365,0.661363,16.290984,0.661365,0,1
3,Relaxed1,Left,Gyroscope,X,0.652301,0.652300,16.290984,0.652301,0,1
4,Relaxed1,Left,Gyroscope,Y,0.648110,0.648110,15.778689,0.648110,0,1
...,...,...,...,...,...,...,...,...,...,...
61903,Entrainment2,Right,Accelerometer,Y,0.509812,0.509802,11.372951,0.509812,0,469
61904,Entrainment2,Right,Accelerometer,Z,0.519732,0.519716,11.372951,0.519732,0,469
61905,Entrainment2,Right,Gyroscope,X,0.526807,0.526778,11.475410,0.526807,0,469
61906,Entrainment2,Right,Gyroscope,Y,0.527520,0.527478,11.475410,0.527520,0,469


In [65]:
result_df.shape

(61908, 10)

In [71]:
result_df['TremorRating'].value_counts()

TremorRating
0    61411
1      497
Name: count, dtype: int64

In [73]:
tremor_rating_1_ids = result_df[result_df['TremorRating'] == 1]['ID']

In [87]:
tremor_rating_1_ids_list = tremor_rating_1_ids.tolist()

In [93]:
#Filtering the DataFrame for TremorRating of 1
tremor_rating_1_ids = result_df[result_df['TremorRating'] == 1]['ID']

# Getting distinct IDs
distinct_tremor_rating_1_ids = tremor_rating_1_ids.unique()

# Converting to a list
distinct_tremor_rating_1_ids_list = distinct_tremor_rating_1_ids.tolist()
print(distinct_tremor_rating_1_ids_list)

[60, 68, 74, 79, 102, 228, 297, 327, 384, 442, 458]


In [61]:
df1=result_df[result_df['ID']==1]

In [53]:
df1.shape

(132, 10)

In [55]:
df1['TremorRating'].value_counts()

TremorRating
0    132
Name: count, dtype: int64

In [59]:
df2=result_df[result_df['ID']==2]

In [63]:
df2

,Movement,Side,Sensor,Axis,RMS,STD,Frequency,Amplitude,TremorRating,ID
132,Relaxed1,Left,Accelerometer,X,0.462714,0.462651,38.319672,0.462714,0,2
133,Relaxed1,Left,Accelerometer,Y,0.446852,0.446842,38.524590,0.446852,0,2
134,Relaxed1,Left,Accelerometer,Z,0.441988,0.441978,39.959016,0.441988,0,2
135,Relaxed1,Left,Gyroscope,X,0.442497,0.442393,40.061475,0.442497,0,2
136,Relaxed1,Left,Gyroscope,Y,0.443289,0.443004,40.061475,0.443289,0,2
...,...,...,...,...,...,...,...,...,...,...
259,Entrainment2,Right,Accelerometer,Y,0.537286,0.537134,38.319672,0.537286,0,2
260,Entrainment2,Right,Accelerometer,Z,0.540967,0.540838,38.319672,0.540967,0,2
261,Entrainment2,Right,Gyroscope,X,0.523117,0.522941,38.319672,0.523117,0,2
262,Entrainment2,Right,Gyroscope,Y,0.495845,0.495663,38.319672,0.495845,0,2


In [95]:
import numpy as np
import pandas as pd
from scipy.fft import fft

def calculate_rms(signal):
    """Calculate the Root Mean Square (RMS) value of the signal."""
    return np.sqrt(np.mean(np.square(signal)))

def calculate_std(signal):
    """Calculate standard deviation of the signal."""
    return np.std(signal)

def calculate_dominant_frequency(signal, fs=100):
    """Calculate the dominant frequency of the signal using FFT."""
    n = len(signal)
    freqs = np.fft.fftfreq(n, 1/fs)
    fft_vals = fft(signal)
    positive_freqs = freqs[:n//2]
    positive_fft_vals = np.abs(fft_vals[:n//2])
    dominant_freq = positive_freqs[np.argmax(positive_fft_vals)]
    return dominant_freq

def convert_to_displacement(signal, sensor_type, dominant_freq, radius=10):
    """Convert acceleration (g) and angular velocity (rad/s) to displacement (cm)."""
    
    if sensor_type == "Accelerometer":
        # Convert g to cm/s² and compute displacement
        acceleration_cm_s2 = signal * 980  # Convert g to cm/s²
        if dominant_freq > 0:
            displacement_cm = acceleration_cm_s2 / (4 * np.pi**2 * dominant_freq**2)
        else:
            displacement_cm = 0  # Avoid division by zero
    elif sensor_type == "Gyroscope":
        # Assume rotational tremor with a given radius (e.g., 10 cm for hand tremor)
        displacement_cm = signal * radius  # Approximate displacement
    else:
        displacement_cm = 0  # Unknown sensor type

    return np.abs(displacement_cm)  # Ensure positive values

def calculate_tremor_rating(displacement_cm):
    """Calculate the tremor rating based on computed displacement in cm."""
    if displacement_cm < 1:
        return 0  # Normal
    elif displacement_cm < 3:
        return 1  # Slight
    elif displacement_cm < 10:
        return 2  # Mild
    elif displacement_cm < 20:
        return 3  # Moderate
    else:
        return 4  # Severe

def process_movement(df, movement_name, side, sensor_type, axis):
    """Process a single movement (e.g., Relaxed1, StretchHold) for a given side, sensor, and axis."""
    column_name = f"{movement_name}_{side}_{sensor_type}_{axis}"
    
    if column_name not in df.columns:
        return None  # Skip if column is missing

    signal = df[column_name].values

    # Compute features
    rms_value = calculate_rms(signal)
    std_value = calculate_std(signal)
    freq_value = calculate_dominant_frequency(signal, fs=100)
    displacement_cm = convert_to_displacement(rms_value, sensor_type, freq_value)  # Convert to cm
    tremor_rating = calculate_tremor_rating(displacement_cm)

    return {
        "ID": df["ID"].iloc[0],
        "Movement": movement_name,
        "Side": side,
        "Sensor": sensor_type,
        "Axis": axis,
        "RMS": rms_value,
        "STD": std_value,
        "Frequency": freq_value,
        "Amplitude (cm)": displacement_cm,
        "TremorRating": tremor_rating
    }

def process_all_patients(df):
    """Process all patients (IDs 1 to 469) and all movements."""
    results = []

    movements = ['Relaxed1', 'Relaxed2', 'RelaxedTask1', 'RelaxedTask2', 'StretchHold', 'HoldWeight', 
                 'DrinkGlas', 'CrossArms', 'TouchNose', 'Entrainment1', 'Entrainment2']
    sides = ['Left', 'Right']
    sensor_types = ['Accelerometer', 'Gyroscope']
    axes = ['X', 'Y', 'Z']
    
    for patient_id in df['ID'].unique():
        patient_data = df[df['ID'] == patient_id]
        
        for movement in movements:
            for side in sides:
                for sensor_type in sensor_types:
                    for axis in axes:
                        result = process_movement(patient_data, movement, side, sensor_type, axis)
                        if result:  # Only add if valid
                            results.append(result)
    
    return pd.DataFrame(results)

# Example usage
# Assuming 'df' is your DataFrame with the necessary columns
result_df = process_all_patients(df)
print(result_df)


        ID      Movement   Side         Sensor Axis       RMS       STD  \
0        1      Relaxed1   Left  Accelerometer    X  0.669875  0.669874   
1        1      Relaxed1   Left  Accelerometer    Y  0.666545  0.666544   
2        1      Relaxed1   Left  Accelerometer    Z  0.661365  0.661363   
3        1      Relaxed1   Left      Gyroscope    X  0.652301  0.652300   
4        1      Relaxed1   Left      Gyroscope    Y  0.648110  0.648110   
...    ...           ...    ...            ...  ...       ...       ...   
61903  469  Entrainment2  Right  Accelerometer    Y  0.509812  0.509802   
61904  469  Entrainment2  Right  Accelerometer    Z  0.519732  0.519716   
61905  469  Entrainment2  Right      Gyroscope    X  0.526807  0.526778   
61906  469  Entrainment2  Right      Gyroscope    Y  0.527520  0.527478   
61907  469  Entrainment2  Right      Gyroscope    Z  0.521691  0.521638   

       Frequency  Amplitude (cm)  TremorRating  
0      16.290984        0.062656             0  
1

In [97]:
result_df

,ID,Movement,Side,Sensor,Axis,RMS,STD,Frequency,Amplitude (cm),TremorRating
0,1,Relaxed1,Left,Accelerometer,X,0.669875,0.669874,16.290984,0.062656,0
1,1,Relaxed1,Left,Accelerometer,Y,0.666545,0.666544,16.290984,0.062345,0
2,1,Relaxed1,Left,Accelerometer,Z,0.661365,0.661363,16.290984,0.061860,0
3,1,Relaxed1,Left,Gyroscope,X,0.652301,0.652300,16.290984,6.523006,2
4,1,Relaxed1,Left,Gyroscope,Y,0.648110,0.648110,15.778689,6.481101,2
...,...,...,...,...,...,...,...,...,...,...
61903,469,Entrainment2,Right,Accelerometer,Y,0.509812,0.509802,11.372951,0.097843,0
61904,469,Entrainment2,Right,Accelerometer,Z,0.519732,0.519716,11.372951,0.099747,0
61905,469,Entrainment2,Right,Gyroscope,X,0.526807,0.526778,11.475410,5.268073,2
61906,469,Entrainment2,Right,Gyroscope,Y,0.527520,0.527478,11.475410,5.275197,2


In [101]:
result_df['TremorRating'].value_counts()

TremorRating
2    28086
0    28003
1     4624
3      665
4      530
Name: count, dtype: int64

In [103]:
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt, welch
from scipy.fft import fft

def bandpass_filter(signal, lowcut=3, highcut=12, fs=100, order=4):
    """Apply a Butterworth bandpass filter to remove unwanted frequencies."""
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)

def calculate_rms(signal):
    """Calculate the Root Mean Square (RMS) value of the signal."""
    return np.sqrt(np.mean(np.square(signal)))

def calculate_std(signal):
    """Calculate standard deviation of the signal."""
    return np.std(signal)

def calculate_dominant_frequency(signal, fs=100):
    """Calculate the dominant frequency of the signal using Welch's method."""
    freqs, psd = welch(signal, fs=fs, nperseg=fs)
    dominant_freq = freqs[np.argmax(psd)]
    return dominant_freq

def calculate_tremor_rating(amplitude):
    """Calculate the tremor rating based on amplitude in cm."""
    if amplitude < 1:
        return 0  # Normal
    elif amplitude < 3:
        return 1  # Slight
    elif amplitude < 10:
        return 2  # Mild
    elif amplitude < 20:
        return 3  # Moderate
    else:
        return 4  # Severe

def process_movement(df, movement_name, side, sensor_type, axis):
    """Process a single movement (e.g., Relaxed1, StretchHold) for a given side, sensor, and axis."""
    column_name = f"{movement_name}_{side}_{sensor_type}_{axis}"
    
    if column_name not in df.columns:
        return None  # Skip if the column is missing
    
    signal = df[column_name].values

    # Apply bandpass filter
    filtered_signal = bandpass_filter(signal)

    # Calculate features
    rms_value = calculate_rms(filtered_signal)
    std_value = calculate_std(filtered_signal)
    freq_value = calculate_dominant_frequency(filtered_signal, fs=100)
    
    # Adjust amplitude scale if needed
    amplitude_value = rms_value if "Accelerometer" in sensor_type else rms_value * 10  # Adjust scaling factor if needed
    
    tremor_rating = calculate_tremor_rating(amplitude_value)

    return {
        "Movement": movement_name,
        "Side": side,
        "Sensor": sensor_type,
        "Axis": axis,
        "RMS": rms_value,
        "STD": std_value,
        "Frequency": freq_value,
        "Amplitude (cm)": amplitude_value,
        "TremorRating": tremor_rating
    }

def process_all_patients(df):
    """Process all patients (IDs 1 to 469) and all movements."""
    results = []

    movements = ['Relaxed1', 'Relaxed2', 'RelaxedTask1', 'RelaxedTask2', 'StretchHold', 'HoldWeight', 'DrinkGlas', 
                 'CrossArms', 'TouchNose', 'Entrainment1', 'Entrainment2']
    sides = ['Left', 'Right']
    sensor_types = ['Accelerometer', 'Gyroscope']
    axes = ['X', 'Y', 'Z']
    
    for patient_id in df['ID'].unique():
        patient_data = df[df['ID'] == patient_id]
        
        for movement in movements:
            for side in sides:
                for sensor_type in sensor_types:
                    for axis in axes:
                        result = process_movement(patient_data, movement, side, sensor_type, axis)
                        if result:
                            result['ID'] = patient_id
                            results.append(result)
    
    return pd.DataFrame(results)

# Example usage:
# Assuming 'df' is your DataFrame with the necessary columns
result_df = process_all_patients(df)
print(result_df)


           Movement   Side         Sensor Axis       RMS       STD  Frequency  \
0          Relaxed1   Left  Accelerometer    X  0.267502  0.267502        9.0   
1          Relaxed1   Left  Accelerometer    Y  0.259317  0.259317        9.0   
2          Relaxed1   Left  Accelerometer    Z  0.257116  0.257116        9.0   
3          Relaxed1   Left      Gyroscope    X  0.253317  0.253317        9.0   
4          Relaxed1   Left      Gyroscope    Y  0.253939  0.253939        9.0   
...             ...    ...            ...  ...       ...       ...        ...   
61903  Entrainment2  Right  Accelerometer    Y  0.302304  0.302303        9.0   
61904  Entrainment2  Right  Accelerometer    Z  0.298851  0.298851        9.0   
61905  Entrainment2  Right      Gyroscope    X  0.292959  0.292959        9.0   
61906  Entrainment2  Right      Gyroscope    Y  0.286526  0.286526        8.0   
61907  Entrainment2  Right      Gyroscope    Z  0.282438  0.282438        8.0   

       Amplitude (cm)  Trem

In [105]:
result_df

,Movement,Side,Sensor,Axis,RMS,STD,Frequency,Amplitude (cm),TremorRating,ID
0,Relaxed1,Left,Accelerometer,X,0.267502,0.267502,9.0,0.267502,0,1
1,Relaxed1,Left,Accelerometer,Y,0.259317,0.259317,9.0,0.259317,0,1
2,Relaxed1,Left,Accelerometer,Z,0.257116,0.257116,9.0,0.257116,0,1
3,Relaxed1,Left,Gyroscope,X,0.253317,0.253317,9.0,2.533167,1,1
4,Relaxed1,Left,Gyroscope,Y,0.253939,0.253939,9.0,2.539390,1,1
...,...,...,...,...,...,...,...,...,...,...
61903,Entrainment2,Right,Accelerometer,Y,0.302304,0.302303,9.0,0.302304,0,469
61904,Entrainment2,Right,Accelerometer,Z,0.298851,0.298851,9.0,0.298851,0,469
61905,Entrainment2,Right,Gyroscope,X,0.292959,0.292959,9.0,2.929588,1,469
61906,Entrainment2,Right,Gyroscope,Y,0.286526,0.286526,8.0,2.865260,1,469


In [107]:
result_df['TremorRating'].value_counts()

TremorRating
0    36689
1    19768
2     5451
Name: count, dtype: int64

In [109]:
tremor_rating_1_ids = result_df[result_df['TremorRating'] == 1]['ID']

In [111]:
tremor_rating_1_ids

3          1
4          1
5          1
123        1
124        1
        ... 
61900    469
61901    469
61905    469
61906    469
61907    469
Name: ID, Length: 19768, dtype: int64

In [113]:
#Filtering the DataFrame for TremorRating of 1
tremor_rating_1_ids = result_df[result_df['TremorRating'] == 1]['ID']

# Getting distinct IDs
distinct_tremor_rating_1_ids = tremor_rating_1_ids.unique()

# Converting to a list
distinct_tremor_rating_1_ids_list = distinct_tremor_rating_1_ids.tolist()
print(distinct_tremor_rating_1_ids_list)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 158, 159, 160, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 222, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 234, 23